In [2]:
import pandas as pd

# Đọc dataset
df = pd.read_csv("DATA/all_stocks_5yr.csv")

# Lọc chỉ 3 cổ phiếu mong muốn
filtered_df = df[df['Name'].isin(['AAL', 'ADP', 'AJG'])]

# Xem trước dữ liệu
print(filtered_df.head())

# Nếu muốn lưu lại file mới:
filtered_df.to_csv("DATA/stocks_AAL_ADP_AJG.csv", index=False)

# lưu thành file stock.csv
filtered_df.to_csv("DATA/stock.csv", index=False)

         date   open   high    low  close    volume Name
0  2013-02-08  15.07  15.12  14.63  14.75   8407500  AAL
1  2013-02-11  14.89  15.01  14.26  14.46   8882000  AAL
2  2013-02-12  14.45  14.51  14.10  14.27   8126000  AAL
3  2013-02-13  14.30  14.94  14.25  14.66  10259500  AAL
4  2013-02-14  14.94  14.96  13.16  13.99  31879900  AAL


In [7]:
# train_forecast_7day.py
"""
Script train và so sánh 2 model deep RNN (SimpleRNN) và LSTM (mỗi model >=7 recurrent layers),
vẽ loss + MAE, và dự báo NGẮN 7 ngày tiếp theo (iterative forecasting).
Input CSV: DATA/stock.csv (cột: date, open, high, low, close, volume, Name)
Output lưu vào folder models_and_results/
"""
# Nguyễn Việt Quang B22DCCN650
import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# ---------------- CONFIG ----------------
# Nguyễn Việt Quang B22DCCN650
CSV_PATH = "DATA/stock.csv"
OUTPUT_DIR = "models_and_results"
SEQ_LEN = 60           # số ngày lịch sử dùng làm input
HORIZON = 7            # dự báo 7 ngày tiếp theo
BATCH_SIZE = 64
EPOCHS = 15
VALID_SPLIT = 0.1
RANDOM_SEED = 42
# ----------------------------------------
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------- util functions ----------------
# Nguyễn Việt Quang B22DCCN650
def load_data(path):
    df = pd.read_csv(path, parse_dates=["date"])
    df.sort_values(["Name", "date"], inplace=True)
    return df

def scale_array(arr):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled = scaler.fit_transform(arr.reshape(-1,1)).flatten()
    return scaled, scaler

def create_sequences(values, seq_len, horizon=1):
    """Tạo sequences; mặc định tạo label 1-step (dùng iterative để dự báo multi-step)."""
    X, y = [], []
    n = len(values)
    for i in range(n - seq_len - horizon + 1):
        X.append(values[i:i+seq_len])
        y.append(values[i+seq_len:i+seq_len+horizon])
    X = np.array(X).reshape(-1, seq_len, 1)
    y = np.array(y)
    return X, y

def build_deep_rnn(input_shape, horizon):
    # 7 recurrent layers với units giảm dần
    units = [128, 128, 64, 64, 32, 32, 16]
    model = Sequential()
    for i, u in enumerate(units):
        if i == 0:
            model.add(SimpleRNN(u, return_sequences=True, input_shape=input_shape))
        elif i < len(units)-1:
            model.add(SimpleRNN(u, return_sequences=True))
        else:
            model.add(SimpleRNN(u, return_sequences=False))
        model.add(BatchNormalization())
        model.add(Dropout(0.15))
    model.add(Dense(horizon))
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Nguyên Việt Quang B22DCCN650
def build_deep_lstm(input_shape, horizon):
    units = [128, 128, 64, 64, 32, 32, 16]
    model = Sequential()
    for i, u in enumerate(units):
        if i == 0:
            model.add(LSTM(u, return_sequences=True, input_shape=input_shape))
        elif i < len(units)-1:
            model.add(LSTM(u, return_sequences=True))
        else:
            model.add(LSTM(u, return_sequences=False))
        model.add(BatchNormalization())
        model.add(Dropout(0.15))
    model.add(Dense(horizon))
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model
# Nguyễn Việt Quang B22DCCN650
def plot_history(history, prefix, outdir):
    # Loss
    plt.figure(figsize=(8,4))
    plt.plot(history.history["loss"], label="train_loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.title(f"{prefix} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{prefix}_loss.png"))
    plt.close()

    # MAE
    plt.figure(figsize=(8,4))
    if "mae" in history.history:
        plt.plot(history.history["mae"], label="train_mae")
    if "val_mae" in history.history:
        plt.plot(history.history["val_mae"], label="val_mae")
    plt.title(f"{prefix} - MAE")
    plt.xlabel("Epoch")
    plt.ylabel("MAE")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{prefix}_mae.png"))
    plt.close()

# Nguyễn Việt Quang B22DCCN650
# hàm này có chức năng là dự báo nhiều bước (multi-step) bằng cách sử dụng dự báo từng bước (1-step) lặp đi lặp lại
def iterative_forecast(model, last_sequence, steps, scaler):
    """
    last_sequence: 1D array scaled length SEQ_LEN
    steps: number of iterative predicted steps
    returns: unscaled predictions (1D array length steps)
    """
    seq = list(last_sequence)
    preds_scaled = []
    for _ in range(steps):
        x = np.array(seq[-SEQ_LEN:]).reshape(1, SEQ_LEN, 1)
        p = model.predict(x, verbose=0)[0][0]   # assume model trained 1-step output
        preds_scaled.append(p)
        seq.append(p)
    preds_scaled = np.array(preds_scaled).reshape(-1,1)
    preds = scaler.inverse_transform(preds_scaled).flatten()
    return preds

# ---------------- training + evaluation ----------------
# Nguyễn Việt Quang B22DCCN650
def train_and_save(X, y, builder, prefix):
    n = X.shape[0]
    val_n = max(1, int(n * VALID_SPLIT))
    train_X, val_X = X[:-val_n], X[-val_n:]
    train_y, val_y = y[:-val_n], y[-val_n:]

    model = builder((SEQ_LEN,1), train_y.shape[1])
    ckpt_path = os.path.join(OUTPUT_DIR, f"{prefix}_best.h5")
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1),
        ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1)
    ]
    history = model.fit(
        train_X, train_y,
        validation_data=(val_X, val_y),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=2
    )
    # Save history plots
    plot_history(history, prefix, OUTPUT_DIR)

    # Nguyễn Việt Quang B22DCCN650
    # Evaluate on validation (use first step since y shape is (n,1) here)
    val_pred = model.predict(val_X)
    mse = mean_squared_error(val_y[:,0], val_pred[:,0])
    mae = mean_absolute_error(val_y[:,0], val_pred[:,0])
    metrics = {"mse": float(mse), "mae": float(mae), "val_samples": len(val_X)}
    # return model and metrics
    return model, history, metrics

# ---------------- main pipeline ----------------
# Nguyễn Việt Quang B22DCCN650
def run_pipeline():
    df = load_data(CSV_PATH)
    symbols = df["Name"].unique()
    print("Found symbols:", symbols)
    summary = {}

    for sym in symbols:
        print("\n========================")
        print("Processing:", sym)
        print("========================")
        sub = df[df["Name"] == sym].sort_values("date")
        prices = sub["close"].values.astype(float)
        dates = sub["date"].values

        # scale
        scaled, scaler = scale_array(prices)

        # create sequences for 1-step training (we use iterative forecasting for HORIZON steps)
        X, y = create_sequences(scaled, SEQ_LEN, horizon=1)  # y shape (n,1)

        # Nguyễn Việt Quang B22DCCN650
        # sanity check: enough samples
        if X.shape[0] < 10:
            print(f"Not enough samples for {sym} (need >=10 sequences). Skipping.")
            continue

        # Train RNN
        rnn_prefix = f"{sym}_RNN"
        rnn_model, rnn_hist, rnn_metrics = train_and_save(X, y, build_deep_rnn, rnn_prefix)
        print(f"RNN metrics: {rnn_metrics}")

        # Train LSTM
        lstm_prefix = f"{sym}_LSTM"
        lstm_model, lstm_hist, lstm_metrics = train_and_save(X, y, build_deep_lstm, lstm_prefix)
        print(f"LSTM metrics: {lstm_metrics}")

        # choose best by val MSE
        chosen = "RNN" if rnn_metrics["mse"] <= lstm_metrics["mse"] else "LSTM"
        chosen_model = rnn_model if chosen == "RNN" else lstm_model
        chosen_metrics = rnn_metrics if chosen == "RNN" else lstm_metrics
        print(f"Chosen model for {sym}: {chosen} (val_mse={chosen_metrics['mse']:.6f})")

        # Forecast HORIZON days from last sequence
        last_seq = scaled[-SEQ_LEN:]
        preds = iterative_forecast(chosen_model, last_seq, HORIZON, scaler)

        # Nguyễn Việt Quang B22DCCN650
        # Save results
        out = {
            "symbol": sym,
            "last_date": str(dates[-1]),
            "horizon_days": HORIZON,
            "chosen_model": chosen,
            "chosen_metrics": chosen_metrics,
            "predictions": preds.tolist()
        }
        out_path = os.path.join(OUTPUT_DIR, f"{sym}_forecast_7day.json")
        with open(out_path, "w") as f:
            json.dump(out, f, indent=2)
        summary[sym] = out

        # Optional: save simple plot of recent actual + 7-day forecast
        try:
            recent_n = min(200, len(prices))
            recent_prices = prices[-recent_n:]
            recent_dates = pd.to_datetime(dates[-recent_n:])
            plt.figure(figsize=(10,4))
            plt.plot(recent_dates, recent_prices, label="actual_close")
            # Nguyễn Việt Quang B22DCCN650
            # forecast dates: next calendar days (not trading-day aware)
            last_date = pd.to_datetime(dates[-1])
            fut_dates = [last_date + np.timedelta64(i+1, 'D') for i in range(HORIZON)]
            plt.plot(fut_dates, preds, marker='o', linestyle='--', label=f"{HORIZON}-day forecast")
            plt.title(f"{sym} recent close + {HORIZON}-day forecast")
            plt.xlabel("date")
            plt.ylabel("close")
            plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, f"{sym}_recent_plus_7day.png"))
            plt.close()
        except Exception as e:
            print("Plot save skipped:", e)

    # write summary
    with open(os.path.join(OUTPUT_DIR, "summary_7day.json"), "w") as f:
        json.dump(summary, f, indent=2)
    print("All done. Results saved to", OUTPUT_DIR)

if __name__ == "__main__":
    run_pipeline()


Found symbols: ['AAL' 'ADP' 'AJG']

Processing: AAL


D:\school\se1_year4\IntSys\tieu_luan\code\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15

Epoch 1: val_loss improved from None to 0.14052, saving model to models_and_results\AAL_RNN_best.h5


17/17 - 17s - 999ms/step - loss: 1.6074 - mae: 1.0179 - val_loss: 0.1405 - val_mae: 0.3659
Epoch 2/15

Epoch 2: val_loss did not improve from 0.14052
17/17 - 2s - 138ms/step - loss: 1.0784 - mae: 0.8346 - val_loss: 0.7890 - val_mae: 0.8843
Epoch 3/15

Epoch 3: val_loss improved from 0.14052 to 0.04879, saving model to models_and_results\AAL_RNN_best.h5


17/17 - 3s - 148ms/step - loss: 0.7669 - mae: 0.6994 - val_loss: 0.0488 - val_mae: 0.1729
Epoch 4/15

Epoch 4: val_loss did not improve from 0.04879
17/17 - 2s - 134ms/step - loss: 0.6103 - mae: 0.6170 - val_loss: 0.2725 - val_mae: 0.5150
Epoch 5/15

Epoch 5: val_loss did not improve from 0.04879
17/17 - 2s - 139ms/step - loss: 0.5516 - mae: 0.5887 - val_loss: 0.6223 - val_mae: 0.7830
Epoch 6/15

Epoch 6: val_loss did not improve from 0.04879
17/17 - 2s - 135ms/step - loss: 0.4975 - mae: 0.5701 - val_loss: 0.5786 - val_mae: 0.7565
Epoch 7/15

Epoch 7: val_loss did not improve from 0.04879
17/17 - 2s - 136ms/step - loss: 0.3837 - mae: 0.4935 - val_loss: 0.1267 - val_mae: 0.3441
Epoch 8/15

Epoch 8: val_loss did not improve from 0.04879
17/17 - 2s - 142ms/step - loss: 0.3920 - mae: 0.4970 - val_loss: 0.0982 - val_mae: 0.3026
Epoch 9/15

Epoch 9: val_loss did not improve from 0.04879
17/17 - 2s - 134ms/step - loss: 0.3538 - mae: 0.4759 - val_loss: 0.0896 - val_mae: 0.2876
Epoch 10/15

Epo

...

Epoch 1/15

Epoch 1: val_loss improved from None to 0.18014, saving model to models_and_results\AJG_LSTM_best.h5


17/17 - 23s - 1s/step - loss: 0.9460 - mae: 0.7245 - val_loss: 0.1801 - val_mae: 0.4179
Epoch 2/15

Epoch 2: val_loss improved from 0.18014 to 0.07026, saving model to models_and_results\AJG_LSTM_best.h5


17/17 - 6s - 356ms/step - loss: 0.4764 - mae: 0.5155 - val_loss: 0.0703 - val_mae: 0.2534
Epoch 3/15

Epoch 3: val_loss improved from 0.07026 to 0.03512, saving model to models_and_results\AJG_LSTM_best.h5


17/17 - 6s - 356ms/step - loss: 0.3753 - mae: 0.4593 - val_loss: 0.0351 - val_mae: 0.1693
Epoch 4/15

Epoch 4: val_loss improved from 0.03512 to 0.02185, saving model to models_and_results\AJG_LSTM_best.h5


17/17 - 6s - 358ms/step - loss: 0.3371 - mae: 0.4368 - val_loss: 0.0218 - val_mae: 0.1285
Epoch 5/15

Epoch 5: val_loss did not improve from 0.02185
17/17 - 6s - 356ms/step - loss: 0.3360 - mae: 0.4297 - val_loss: 0.0220 - val_mae: 0.1286
Epoch 6/15

Epoch 6: val_loss did not improve from 0.02185
17/17 - 6s - 351ms/step - loss: 0.2986 - mae: 0.4252 - val_loss: 0.0568 - val_mae: 0.2201
Epoch 7/15

Epoch 7: val_loss did not improve from 0.02185
17/17 - 6s - 354ms/step - loss: 0.2811 - mae: 0.4103 - val_loss: 0.0896 - val_mae: 0.2867
Epoch 8/15

Epoch 8: val_loss did not improve from 0.02185
17/17 - 6s - 348ms/step - loss: 0.2743 - mae: 0.4017 - val_loss: 0.1326 - val_mae: 0.3543
Epoch 9/15

Epoch 9: val_loss did not improve from 0.02185
17/17 - 6s - 370ms/step - loss: 0.2593 - mae: 0.3909 - val_loss: 0.1210 - val_mae: 0.3381
Epoch 10/15

Epoch 10: val_loss did not improve from 0.02185
17/17 - 6s - 353ms/step - loss: 0.2368 - mae: 0.3589 - val_loss: 0.1351 - val_mae: 0.3585
Epoch 11/15

E